In [1]:
# Jupyter Notebook cell: R-GCN training and evaluation

import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from torch.utils.data import DataLoader
from torch_geometric.data import Data
from torch_geometric.nn import RGCNConv
from tqdm.auto import tqdm

c:\Users\jerry\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:


# 1. Load triples
def load_triples(path):
    triples = []
    with open(path, 'r') as f:
        for line in f:
            h, r, t = line.strip().split()[:3]
            triples.append((h, r, t))
    return triples

train_triples = load_triples("../graph_data/train.tsv")
dev_triples =  load_triples("../graph_data/dev.tsv")
test_triples =  load_triples("../graph_data/test.tsv")

# 2. Build vocab and graph edges
entities = sorted({e for (h, _, t) in train_triples + dev_triples + test_triples for e in (h, t)})
relations = sorted({r for (_, r, _) in train_triples + dev_triples + test_triples})
ent2id = {e: i for i, e in enumerate(entities)}
rel2id = {r: i for i, r in enumerate(relations)}
num_nodes = len(entities)
num_rels = len(relations)

# Build edge index and edge types
edge_index = [[], []]
edge_type = []
for h, r, t in train_triples:
    src = ent2id[h]
    dst = ent2id[t]
    edge_index[0].append(src)
    edge_index[1].append(dst)
    edge_type.append(rel2id[r])
# convert to tensors
edge_index = torch.tensor(edge_index, dtype=torch.long)
edge_type = torch.tensor(edge_type, dtype=torch.long)



data = Data(num_nodes=num_nodes, edge_index=edge_index, edge_type=edge_type)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

data = data.to(device)

In [3]:
# 3. Model definition
class RGCN(nn.Module):
    def __init__(self, num_nodes, num_rels, hidden_dim, num_layers=2, dropout=0.3):
        super().__init__()
        self.convs = nn.ModuleList()
        # input to first layer
        self.convs.append(RGCNConv(num_nodes, hidden_dim, num_rels, num_bases=30))
        for _ in range(num_layers-1):
            self.convs.append(RGCNConv(hidden_dim, hidden_dim, num_rels, num_bases=30))
        self.dropout = dropout
        # relation embeddings for scoring
        self.rel_emb = nn.Embedding(num_rels, hidden_dim)
        nn.init.xavier_uniform_(self.rel_emb.weight)

    def forward(self, x, edge_index, edge_type):
        for conv in self.convs:
            x = conv(x, edge_index, edge_type)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        return x

# 4. Negative sampling for triples

def negative_sampling(batch, num_entities):
    neg = batch.clone()
    mask = torch.rand(batch.size(0), device=batch.device) < 0.5
    num_h = mask.sum().item()
    neg[mask, 0] = torch.randint(0, num_entities, (num_h,), device=batch.device)
    num_t = (~mask).sum().item()
    neg[~mask, 2] = torch.randint(0, num_entities, (num_t,), device=batch.device)
    return neg

# 5. Encode triples into index tensors

def encode(triples):
    return torch.tensor([[ent2id[h], rel2id[r], ent2id[t]] for h, r, t in triples], dtype=torch.long)

In [ ]:
train_data = encode(train_triples).to(device)
dev_data   = encode(dev_triples).to(device)
test_data  = encode(test_triples).to(device)

# 6. Instantiate model and optimizer
model = RGCN(num_nodes, num_rels, hidden_dim=100, num_layers=2).to(device)
optimizer = AdamW(model.parameters(), lr=5e-4, weight_decay=1e-4)


In [5]:
# 7. Training and evaluation functions

def score_triples(node_emb, triples):
    hs, rs, ts = triples[:,0], triples[:,1], triples[:,2]
    re = node_emb[hs] * model.rel_emb(rs)
    score = (re * node_emb[ts]).sum(dim=-1)
    return score

def train_epoch():
    model.train()
    total_loss = 0; count=0
    embeddings = torch.eye(num_nodes, device=device)  # one-hot initial
    for batch in tqdm(DataLoader(train_data, batch_size=1024, shuffle=True), desc='Train', leave=False):
        neg = negative_sampling(batch, num_nodes)
        node_emb = model(embeddings, data.edge_index, data.edge_type)
        pos_score = score_triples(node_emb, batch)
        neg_score = score_triples(node_emb, neg)
        loss = - (F.logsigmoid(pos_score).mean() + F.logsigmoid(-neg_score).mean())
        optimizer.zero_grad(); loss.backward(); optimizer.step()
        total_loss += loss.item() * batch.size(0); count += batch.size(0)
    return total_loss/count

# 9. Evaluation
torch.no_grad()
def evaluate(triples_tensor, num_neg=100, k_list=(1,3,10)):
    model.eval()
    ranks = []
    embeddings = torch.eye(num_nodes, device=device)
    node_emb = model(embeddings, data.edge_index, data.edge_type)
    for idx in tqdm(range(triples_tensor.size(0)), desc='Eval', leave=False):
        triple = triples_tensor[idx].unsqueeze(0)
        h, r, t = triple[0]
        pos_score = score_triples(node_emb, triple)[0]
        scores = [pos_score.item()]
        for _ in range(num_neg):
            if random.random() < 0.5:
                h2 = random.randint(0, num_nodes-1); t2 = t.item()
            else:
                h2 = h.item(); t2 = random.randint(0, num_nodes-1)
            neg_triple = torch.tensor([[h2, r, t2]], device=device)
            neg_score = score_triples(node_emb, neg_triple)[0]
            scores.append(neg_score.item())
        sorted_scores = sorted(scores, reverse=True)
        rank = sorted_scores.index(scores[0]) + 1
        ranks.append(rank)
    ranks = torch.tensor(ranks, dtype=torch.float)
    mrr = (1.0 / ranks).mean().item()
    hits = {f"Hits@{k}": (ranks <= k).float().mean().item() for k in k_list}
    return mrr, hits



In [ ]:
dev_best = 0
for epoch in range(1, 201):
    loss = train_epoch()
    mrr_dev, hits_dev = evaluate(dev_data)
    print(f"Epoch {epoch:02d} | Loss: {loss:.4f} | Dev MRR: {mrr_dev:.4f}, Hits@10: {hits_dev['Hits@10']:.4f}")
    dev_best = max(dev_best, mrr_dev)

Epoch 01 | Loss: 1.3859 | Dev MRR: 0.0656, Hits@10: 0.1739


Epoch 02 | Loss: 1.3830 | Dev MRR: 0.0725, Hits@10: 0.1522


Epoch 03 | Loss: 1.3738 | Dev MRR: 0.0832, Hits@10: 0.2174


Epoch 04 | Loss: 1.3611 | Dev MRR: 0.1158, Hits@10: 0.3043


Epoch 05 | Loss: 1.3494 | Dev MRR: 0.0979, Hits@10: 0.2391


...

Epoch 196 | Loss: 0.1977 | Dev MRR: 0.1800, Hits@10: 0.3478


Epoch 197 | Loss: 0.2016 | Dev MRR: 0.2002, Hits@10: 0.3261


Epoch 198 | Loss: 0.1887 | Dev MRR: 0.1854, Hits@10: 0.3261


Epoch 199 | Loss: 0.1776 | Dev MRR: 0.1955, Hits@10: 0.3478


Epoch 200 | Loss: 0.1911 | Dev MRR: 0.1794, Hits@10: 0.3261


In [14]:
# 9. Final test evaluation
mrr_test, hits_test = evaluate(test_data)
print(f"Test MRR: {mrr_test:.4f}   " + ", ".join([f"{k}: {v:.4f}" for k,v in hits_test.items()]))

Test MRR: 0.1331   Hits@1: 0.0435, Hits@3: 0.1522, Hits@10: 0.3261
